In [1]:
import pandas as pd
import gensim

C:\Users\keiko\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import nltk
from nltk.corpus import stopwords

In [3]:
import string
from string import punctuation

In [4]:
beige = pd.read_csv('beige_books.csv')

In [ ]:
#for removing stop words/stemming words
%%capture
!pip install spacy
!python -m spacy download en
!pip install --no-cache-dir wordcloud
!pip3 install --no-cache-dir -U folium
!pip install --no-cache-dir textblob
from datascience import make_array
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
%matplotlib inline
import folium
import pandas as pd
from IPython.display import HTML, display, IFrame
import folium
import spacy
from wordcloud import WordCloud

## Preprocessing

In [5]:
beige.head()

,ID,Date,District,Sector Heading,Sector Text
0,1,5-20-1970,Atlanta,Summary of Economic Activity,The mood of our directors varies from pessimis...
1,2,5-20-1970,Boston,Summary of Economic Activity,Discussions Monday (May 18) and last Friday wi...
2,3,5-20-1970,Chicago,Summary of Economic Activity,In published statements and private conversati...
3,4,5-20-1970,Cleveland,Summary of Economic Activity,Economic activity in the District has been dis...
4,5,5-20-1970,Dallas,Summary of Economic Activity,Continued inflation is the major concern of El...


In [6]:
print(len(beige['Sector Text']))
texts_series = beige['Sector Text']
texts = list(beige['Sector Text'])

25273


#### Getting rid of null vals, punctuation marks, stop words, and lowercasing

In [29]:
print(len(beige[texts_series.isnull()]))
beige[texts_series.isnull()]

139


,ID,Date,District,Sector Heading,Sector Text
20,21,5-20-1970,San Francisco,Summary for Selected Economic Aspects,NaN
29,30,5-20-1970,San Francisco,Inventories,NaN
30,31,5-20-1970,San Francisco,Both higher and lower inventories are reported.,NaN
92,93,10-14-1970,Atlanta,five-week delivery.,NaN
121,122,12-9-1970,Cleveland,one- and two-year maturity ranges.,NaN
135,136,1-12-1971,Dallas,as a whole.,NaN
216,217,7-21-1971,San Francisco,August 1.,NaN
258,259,11-10-1971,Boston,Regulation Q.,NaN
584,585,2-13-1974,Richmond,will decline.,NaN
628,629,6-12-1974,Dallas,just 6 percent.,NaN


In [7]:
#get rid of empty text records
beige.dropna(axis = 0, how = 'any', inplace = True)
len(beige)

25134

In [31]:
type(beige['Sector Text'][0])

str

In [8]:
#lowercase all records
beige['Sector Text'] = beige['Sector Text'].str.lower()

In [9]:
beige['Sector Text'][0]

"the mood of our directors varies from pessimism to optimism. if any consensus exists, it is that business activity in the sixth district will continue to weaken slightly. many major economic indices should drift downward or mark time for the next two months at least.in the pessimistic vein, a leading department store recently informed us that their sales are even with a year ago, but only because of the addition of branches; unit sales, of course, are lower. the store reported that labor costs were up 8 to 10 percent and profits were off. they were pessimistic about future department store sales and did not anticipate a quick economic recovery. a telephone survey of department stores indicates this conclusion is shared by many retailers throughout the southeast.according to directors' reports, retail sales in some areas have been adversely affected because of a diversion of purchasing power to private schools which have sprung up in response to integration orders. lumber mills in many

In [10]:
import re

In [35]:
#get rid of punctuation marks
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
ex = 'this @(*&is an exmaple text *&^%@^}{(0} to use'
re.sub('[^\s\w]', '', ex)

'this is an exmaple text 0 to use'

In [11]:
beige['Sector Text'].str.replace(r'[^\w\s]', '')
beige['Sector Text'] = beige['Sector Text'].str.replace(r'[^\w\s]', '')

In [ ]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [12]:
beige['District'].unique()

array(['Atlanta', 'Boston', 'Chicago', 'Cleveland', 'Dallas',
       'Kansas City', 'Minneapolis', 'New York', 'Philadelphia',
       'Richmond', 'San Francisco', 'St Louis', 'District'], dtype=object)

In [46]:
#ny = beige[beige['District'] == 'New York']
#ny.to_csv(path_or_buf = 'ny.csv', sep = ',', encoding='utf-8')

In [13]:
toks_tab = Table()
toks_tab.append_column(label="Word", values=[word.text for word in beige['Sector Text'][0]])
toks_tab.show()

NameError: name 'Table' is not defined

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keiko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
#get rid of stop words
stop_words = stopwords.words('english')

In [16]:
#remove stop words
def rem_stop(text):
    
    from nltk.corpus import stopwords
    
    stop_words = stopwords.words("english")
       
    toks = nltk.word_tokenize(text)
    toks_reduced = [x for x in toks if x.lower() not in stopwords.words('english')]
    
    return toks_reduced 

In [56]:
len(beige['Sector Text'])

25134

In [17]:
beige['tokenized'] = [rem_stop(t) for t in beige['Sector Text']]

In [18]:
beige.to_csv(path_or_buf = 'beige_tokenized.csv', sep = ',', encoding='utf-8')

In [20]:
token = pd.read_csv('beige_tokenized.csv')
token.head()



,Unnamed: 0,ID,Date,District,Sector Heading,Sector Text,tokenized
0,0,1,5-20-1970,Atlanta,Summary of Economic Activity,the mood of our directors varies from pessimis...,"['mood', 'directors', 'varies', 'pessimism', '..."
1,1,2,5-20-1970,Boston,Summary of Economic Activity,discussions monday may 18 and last friday with...,"['discussions', 'monday', 'may', '18', 'last',..."
2,2,3,5-20-1970,Chicago,Summary of Economic Activity,in published statements and private conversati...,"['published', 'statements', 'private', 'conver..."
3,3,4,5-20-1970,Cleveland,Summary of Economic Activity,economic activity in the district has been dis...,"['economic', 'activity', 'district', 'disrupte..."
4,4,5,5-20-1970,Dallas,Summary of Economic Activity,continued inflation is the major concern of el...,"['continued', 'inflation', 'major', 'concern',..."


In [ ]:
len(token['tokenized'][0])